In [2]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.0.0rc1

In [3]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.0.0rc1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.0.0rc1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

<div class="alert alert-block alert-danger">This notebook is designed for an older StellarGraph version 1.0.0rc1 and may not function correctly with the newer installed version 1.0.0. Please see: <a href="https://github.com/stellargraph/stellargraph/issues/1172">https://github.com/stellargraph/stellargraph/issues/1172</a>.</div>

C:\Users\mukun\anaconda3\envs\graphemb\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: This notebook is designed for an older StellarGraph version 1.0.0rc1 and may not function correctly with the newer installed version 1.0.0. Please see: <https://github.com/stellargraph/stellargraph/issues/1172>.
  """


In [34]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import random
from pathlib import Path

import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UniformRandomWalk
from stellargraph.data import UnsupervisedSampler
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow as tf
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score

from stellargraph import globalvar

from stellargraph import datasets
from IPython.display import display, HTML

if tf.test.gpu_device_name():
  print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

Default GPU Device:/device:GPU:0


### Loading the protein data

In [5]:
dataset = datasets.PROTEINS()
display(HTML(dataset.description))
G_list, node_subjects_all = dataset.load()

In [48]:
G = G_list[11]
node_subjects = node_subjects_all[G.nodes()]
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 52, Edges: 206

 Node types:
  default: [52]
    Features: float32 vector, length 4
    Edge types: default-default->default

 Edge types:
    default-default->default: [206]
        Weights: all 1 (default)


**1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.**

In [60]:
nodes = list(G.nodes())
number_of_walks = 8 #
length = 2 # Too much

SEED = 0

**2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.**

In [61]:
biased_walker = sg.data.BiasedRandomWalk(G, n=number_of_walks, length=length, p=2, q=.5, seed=SEED)

unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, walker=biased_walker
)

In [62]:
# unsupervised_samples.run(2)

**3. Create a node pair generator:**

Next, create the node pair generator for sampling and streaming the training data to the model. The node pair generator essentially "maps" pairs of nodes `(target, context)` to the input of GraphSAGE: it either takes minibatches of node pairs, or an `UnsupervisedSampler` instance which generates the minibatches of node pairs on demand. The generator samples 2-hop subgraphs with `(target, context)` head nodes extracted from those pairs, and feeds them, together with the corresponding binary labels indicating which pair represent positive or negative sample, to the input layer of the node pair classifier with GraphSAGE node encoder, for SGD updates of the model parameters.

Specify:
1. The minibatch size (number of node pairs per minibatch).
2. The number of epochs for training the model.
3. The sizes of 1- and 2-hop neighbor samples for GraphSAGE:

Note that the length of `num_samples` list defines the number of layers/iterations in the GraphSAGE encoder. In this example, we are defining a 2-layer GraphSAGE encoder.

In [63]:
batch_size = 50
epochs = 20
n_samples = [10, 5]

In the following we show the working of node pair generator with the UnsupervisedSampler, which will generate samples on demand.

In [64]:
generator = GraphSAGELinkGenerator(G, batch_size, num_samples=n_samples, seed=SEED)
train_gen = generator.flow(unsupervised_samples)


Build the model: a 2-layer GraphSAGE encoder acting as node representation learner, with a link classification layer on concatenated (`citing-paper`, `cited-paper`) node embeddings.

GraphSAGE part of the model, with hidden layer sizes of 50 for both GraphSAGE layers, a bias term, and no dropout. (Dropout can be switched on by specifying a positive dropout rate, 0 < dropout < 1).
**Note that the length of `layer_sizes` list must be equal to the length of `num_samples`, as `len(num_samples)` defines the number of hops (layers) in the GraphSAGE encoder**.

In [65]:
layer_sizes = [16, 16]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=generator, bias=True, dropout=0.3, normalize="l2", aggregator=sg.layer.MeanAggregator
)

In [66]:
# Build the model and expose input and output sockets of graphsage, for node pair inputs:
x_inp, x_out = graphsage.in_out_tensors()

In [67]:
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="concat"
)(x_out)

link_classification: using 'concat' method to combine node embeddings into edge embeddings


In [68]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-4),
    loss=keras.losses.binary_crossentropy,
    metrics=['binary_accuracy'],
)

In [69]:
# checkpoint
filepath="./checkpoints/test/weights-improvement-{epoch:02d}-{binary_accuracy:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(str(filepath), 
                             monitor='binary_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True,
                             mode='auto')
callbacks_list = [] # Do not save for this task

**4. Train the model.**

In [70]:
history = model.fit(
    train_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
    callbacks=callbacks_list
)

  ...
    to  
  ['...']
Train for 17 steps
Epoch 1/20
17/17 [==============================] - 2s 96ms/step - loss: 0.6997 - binary_accuracy: 0.5072
Epoch 2/20
17/17 [==============================] - 1s 35ms/step - loss: 0.6990 - binary_accuracy: 0.5000
Epoch 3/20
17/17 [==============================] - 1s 36ms/step - loss: 0.6953 - binary_accuracy: 0.5132
Epoch 4/20
17/17 [==============================] - 1s 33ms/step - loss: 0.7013 - binary_accuracy: 0.4940
Epoch 5/20
17/17 [==============================] - 1s 38ms/step - loss: 0.7035 - binary_accuracy: 0.4772
Epoch 6/20
17/17 [==============================] - 1s 32ms/step - loss: 0.6952 - binary_accuracy: 0.5300
Epoch 7/20
17/17 [==============================] - 1s 32ms/step - loss: 0.7010 - binary_accuracy: 0.5012
Epoch 8/20
17/17 [==============================] - 1s 36ms/step - loss: 0.6946 - binary_accuracy: 0.5288
Epoch 9/20
17/17 [==============================] - 1s 36ms/step - loss: 0.6999 - binary_accuracy: 0.4964
Ep

### Extracting node embeddings
Now that the node pair classifier is trained, we can use its node encoder part as node embeddings evaluator. Below we evaluate node embeddings as activations of the output of graphsage layer stack, and visualise them, coloring nodes by their subject label.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from stellargraph.mapper import GraphSAGENodeGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

**Building a new node-based model**

The `(src, dst)` node pair classifier `model` has two identical node encoders: one for source nodes in the node pairs, the other for destination nodes in the node pairs passed to the model. We can use either of the two identical encoders to evaluate node embeddings. Below we create an embedding model by defining a new Keras model with `x_inp_src` (a list of odd elements in `x_inp`) and `x_out_src` (the 1st element in `x_out`) as input and output, respectively. Note that this model's weights are the same as those of the corresponding node encoder in the previously trained node pair classifier.

In [ ]:
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [ ]:
print(x_inp_src)

We also need a node generator to feed graph nodes to `embedding_model`. We want to evaluate node embeddings for all nodes in the graph:

In [ ]:
df

In [ ]:
node_ids = feature_df.index
node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_ids)

In [ ]:
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

#### Visualize the node embeddings 
Next we visualize the node embeddings in 2D using t-SNE. Colors of the nodes depict their true classes (subject in the case of Cora dataset) of the nodes. 

In [ ]:
# node_subject = node_subjects.astype("category").cat.codes

X = node_embeddings
if X.shape[1] > 2:
    transform = TSNE  # PCA

    trans = transform(n_components=2)
    emb_transformed = pd.DataFrame(trans.fit_transform(X), index=node_ids)
#     emb_transformed["label"] = node_subject
else:
    emb_transformed = pd.DataFrame(X, index=node_ids)
    emb_transformed = emb_transformed.rename(columns={"0": 0, "1": 1})
#     emb_transformed["label"] = node_subject

In [ ]:
alpha = 0.7

fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(
    emb_transformed[0],
    emb_transformed[1],
#     c=emb_transformed["label"].astype("category"),
    cmap="jet",
    alpha=alpha,
)
ax.set(aspect="equal", xlabel="$X_1$", ylabel="$X_2$")
plt.title(
    "{} visualization of GraphSAGE embeddings for cora dataset".format(transform.__name__)
)
plt.show()

In [ ]:
emb_transformed.head()